$\color{blue}{\text{ Assignment 1:  Segmenting and Clustering Neighborhoods in Toronto}}$ 

### 1. Start by creating a new Notebook for this assignment.

In [20]:
#Installing lxml to read the html-file
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4
!pip install geocoder 

     |████████████████████████████████| 102kB 20.0MB/s ta 0:00:01
     |████████████████████████████████| 829kB 37.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 23.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


In [2]:
import pandas as pd
import html5lib
import lxml


### 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(wiki_url) # dfs is a list and its first element is the dataframe
#Toronto_df = pd.concat(dfs)
Toronto_df=dfs[0]
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In the following lines, I check if the table is imported correctly ...

In [4]:
type(Toronto_df)

pandas.core.frame.DataFrame

In [5]:
Toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postcode         288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [6]:
Toronto_df.shape

(288, 3)

### 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
#first 5 rows of the dataframe
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
Toronto_df = Toronto_df[Toronto_df.Borough != 'Not assigned']

In [9]:
Toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
Toronto_df=Toronto_df.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

In [11]:
Toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [12]:
Toronto_df['Neighbourhood'] = Toronto_df.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'] , axis=1)

In [13]:
Toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

To be sure, I checked if there exist any duplicate row in the dataframe

In [14]:
Toronto_df = Toronto_df.drop_duplicates()

I sorted the values of the column alphabetically according to the column Borough

In [28]:
Toronto_df.sort_values('Borough')

,Postcode,Borough,Neighbourhood
47,M4S,Central Toronto,Davisville
45,M4P,Central Toronto,Davisville North
44,M4N,Central Toronto,Lawrence Park
48,M4T,Central Toronto,"Summerhill East, Moore Park"
49,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So..."
64,M5P,Central Toronto,"Forest Hill West, Forest Hill North"
46,M4R,Central Toronto,North Toronto West
63,M5N,Central Toronto,Roselawn
65,M5R,Central Toronto,"Yorkville, The Annex, North Midtown"
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown"


,Postcode,Borough,Neighbourhood
47,M4S,Central Toronto,Davisville
63,M5N,Central Toronto,Roselawn
46,M4R,Central Toronto,North Toronto West
64,M5P,Central Toronto,"Forest Hill West, Forest Hill North"
65,M5R,Central Toronto,"Yorkville, The Annex, North Midtown"
48,M4T,Central Toronto,"Summerhill East, Moore Park"
44,M4N,Central Toronto,Lawrence Park
45,M4P,Central Toronto,Davisville North
49,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So..."
50,M4W,Downtown Toronto,Rosedale


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [17]:
Toronto_df.shape

(103, 3)

- Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [25]:
Geo_df=pd.read_csv("http://cocl.us/Geospatial_data")

In [27]:
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- The column of Toronto_df for the postal code is "Postcode" and for the dataframe Geo_df it is called "Postal Code". So we have to rename it, in order to eb able to merge the both dataframe together

In [35]:
Geo_df=Geo_df.rename(columns={"Postal Code": "Postcode"})
Geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
Toronto_Neighbourhood_df = pd.merge(Toronto_df, Geo_df, on='Postcode')

Toronto_Neighbourhood_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M5P,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307
4,M5R,Central Toronto,"Yorkville, The Annex, North Midtown",43.672710,-79.405678
5,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
6,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
7,M4P,Central Toronto,Davisville North,43.712751,-79.390197
8,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So...",43.686412,-79.400049
9,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
